In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from gan import get_random_generator_input, get_generator_sample
from serialization import load_gan_params, gan_model_dir
from human_pose_util.register import skeleton_register, dataset_register
from human_pose_util.register import register_defaults
from human_pose_util.skeleton import vis3d

In [ ]:
model_name = 'h3m_big'

In [ ]:
register_defaults()
params = load_gan_params(model_name)
skeleton = skeleton_register[
    dataset_register[params['dataset']]['train'].attrs['skeleton_id']]
n_z = params['n_z']

In [ ]:
z_vals = np.zeros((n_z,), dtype=np.float32)

In [ ]:
graph = tf.Graph()
with graph.as_default():
    z = tf.placeholder(shape=(1, n_z,), dtype=tf.float32, name='z')
    sample = get_generator_sample(z, params, reuse=False)
    generator_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

In [ ]:
sess = tf.InteractiveSession(graph=graph)
saver = tf.train.Saver(var_list=generator_vars)
saver.restore(sess, tf.train.latest_checkpoint(gan_model_dir(model_name)))

In [ ]:
@interact(i=range(n_z), z_val=(-3.0, 3.0))
def generate(i=0, z_val=0.):
    z_vals[i] = z_val
    pose = sess.run(sample, feed_dict={z: np.expand_dims(z_vals, axis=0)})[0]
    ax = vis3d(skeleton, pose)
    r = 0.6
    ax.set_zlim3d(0, 2*r)
    ax.set_xlim3d(-r, r)
    ax.set_ylim3d(-r, r)
    plt.show()

In [ ]:
sess.close()